In [2]:
!nvidia-smi

Thu May 30 18:54:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070 Ti     Off | 00000000:01:00.0  On |                  N/A |
|  0%   51C    P3              57W / 310W |    725MiB /  8192MiB |     20%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.32.1 --progress-bar off
!pip install -qqq datasets==2.14.4 --progress-bar off
!pip install -qqq peft==0.5.0 --progress-bar off
!pip install -qqq bitsandbytes==0.41.1 --progress-bar off
!pip install -qqq trl==0.7.1 --progress-bar off
!pip install -qqq spicy==0.16.0 --progress-bar off

^C
ERROR: Operation cancelled by user
^C
ERROR: Operation cancelled by user


In [3]:
import json
import re
from pprint import pprint

import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer

E:\platform\anaconda\envs\deepwork2\Lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [9]:
dataset = load_dataset("Salesforce/dialogstudio", "chitchat-dataset")
dataset

NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

In [5]:
DEFAULT_SYSTEM_PROMPT = """
Below is a conversation between a human and AI agent. Write a summary of the conversation.
""".strip()


def generate_training_prompt(
    conversation: str, summary: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT
) -> str:
    return f""" Instruction: {system_prompt}
### Input: 
{conversation.strip()}

### Response: 
{summary}
""".srtip()

In [7]:
def clean_text(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@[^\s]+", "", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"\^[^ ]+", "", text)
    
def create_conversation_text(data_point):
    text = ""
    for item in data_point["log"]:
        user = clean_etxt(item["user utterance"])
        text += f"user: {user.strip()}\n"
        
        agent = clean_text(item["system response"])
        text += f"agent: {agent.sript()}\n"
        
    return text

In [ ]:
def generate_text(daat_point):
    summaries = json.loads(data_point["original dialog info"])["summaries"][
        "abstractive_summaries"
    ]
    summary = summaries[0]
    summary = " ".join(summary)
    
    conversation_text = create_conversation_text(data_point)
    
    return {
        "conversation": conversation_text,
        "summary": summary,
        "text": generate_training_prompt(conversation_text, summary)
    }